In [511]:
import yfinance as yf
import datetime
import pandas as pd
import numpy as np

In [512]:
start_date = datetime.datetime(2015, 1, 1)
end_date = datetime.datetime(2019, 12, 31)
ticker = "CPHI"
NEUTRAL_BENCHMARK = 0.005

In [513]:
train_data = yf.download(ticker, start=start_date, end=end_date)
test_data = yf.download(ticker, start=end_date, end=datetime.datetime(2023, 11, 29))

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [514]:
print(type(train_data))

<class 'pandas.core.frame.DataFrame'>


In [515]:
train_data["daily_return"] = train_data["Adj Close"].pct_change()
train_data["state"] = np.select(
    [train_data["daily_return"] > NEUTRAL_BENCHMARK, train_data["daily_return"] < -1 * NEUTRAL_BENCHMARK],
    ["up", "down"],
    default="neutral"
)
train_data.head(15)

,Open,High,Low,Close,Adj Close,Volume,daily_return,state
Date,,,,,,,,
2015-01-02,3.1,3.4,3.1,3.1,3.1,2330,NaN,neutral
2015-01-05,3.1,3.5,3.1,3.1,3.1,3890,0.000000,neutral
2015-01-06,3.5,3.5,3.1,3.1,3.1,3040,0.000000,neutral
2015-01-07,3.2,3.5,3.1,3.1,3.1,3600,0.000000,neutral
2015-01-08,3.1,3.4,3.1,3.2,3.2,1810,0.032258,up
2015-01-09,3.3,3.4,3.2,3.2,3.2,200,0.000000,neutral
2015-01-12,3.2,3.3,3.2,3.2,3.2,1140,0.000000,neutral
2015-01-13,3.2,3.3,3.2,3.2,3.2,2420,0.000000,neutral
2015-01-14,3.3,3.3,3.1,3.1,3.1,2010,-0.031250,down


In [516]:
up_to_up = len(train_data[(train_data["state"] == "up") & (train_data["state"].shift(-1) == "up")]) / len(train_data.query('state == "up"'))
up_to_down = len(train_data[(train_data["state"] == "down") & (train_data["state"].shift(-1) == "up")]) / len(train_data.query('state == "up"'))
up_to_neutral = len(train_data[(train_data["state"] == "neutral") & (train_data["state"].shift(-1) == "up")]) / len(train_data.query('state == "up"'))

down_to_down = len(train_data[(train_data["state"] == "down") & (train_data["state"].shift(-1) == "down")]) / len(train_data.query('state == "down"'))
down_to_up = len(train_data[(train_data["state"] == "up") & (train_data["state"].shift(-1) == "down")]) / len(train_data.query('state == "down"'))
down_to_neutral = len(train_data[(train_data["state"] == "neutral") & (train_data["state"].shift(-1) == "down")]) / len(train_data.query('state == "down"'))

neutral_to_neutral = len(train_data[(train_data["state"] == "neutral") & (train_data["state"].shift(-1) == "neutral")]) / len(train_data.query('state == "neutral"'))
neutral_to_up = len(train_data[(train_data["state"] == "up") & (train_data["state"].shift(-1) == "neutral")]) / len(train_data.query('state == "neutral"'))
neutral_to_down = len(train_data[(train_data["state"] == "down") & (train_data["state"].shift(-1) == "neutral")]) / len(train_data.query('state == "neutral"'))

In [517]:
transition_matrix = pd.DataFrame({
    "up": [up_to_up, up_to_neutral, up_to_down],
    "neutral": [neutral_to_up, neutral_to_neutral, neutral_to_down],
    "down": [down_to_up, down_to_neutral, down_to_down]
}, index=["up", "neutral", "down"])
transition_matrix.head()
print(transition_matrix)

               up   neutral      down
up       0.224551  0.203125  0.377129
neutral  0.308383  0.498047  0.374696
down     0.467066  0.296875  0.248175


In [518]:
data.head()

,Open,High,Low,Close,Adj Close,Volume,daily_return,state
Date,,,,,,,,
2020-01-02,74.059998,75.150002,73.797501,75.087502,73.152657,135480400,NaN,down
2020-01-03,74.287498,75.144997,74.125000,74.357498,72.441467,146322800,-0.009722,down
2020-01-06,73.447502,74.989998,73.187500,74.949997,73.018677,118387200,0.007968,up
2020-01-07,74.959999,75.224998,74.370003,74.597504,72.675278,108872000,-0.004703,down
2020-01-08,74.290001,76.110001,74.290001,75.797501,73.844353,132079200,0.016086,up


In [519]:
from backtesting import Backtest, Strategy

In [520]:
from enum import Enum

class Trend(Enum):
    DOWN = "Down"
    UP = "Up"
    NEUTRAL = "Neutral"

In [521]:
max_up_values = max(up_to_up, down_to_up, neutral_to_up)
if max_up_values == up_to_up:
    trend_up = Trend.UP
elif max_up_values == down_to_up:
    trend_up = Trend.DOWN
else:
    trend_up = Trend.NEUTRAL

# Determine trend.down
max_down_values = max(up_to_down, down_to_down, neutral_to_down)
if max_down_values == up_to_down:
    trend_down = Trend.UP
elif max_down_values == down_to_down:
    trend_down = Trend.DOWN
else:
    trend_down = Trend.NEUTRAL

In [522]:
class MarkovStrategy(Strategy):
    def init(self):
        """init"""

    def get_state(self):
        close_yest = self.data.Close[-2]
        close_today = self.data.Close[-1]

        pct = (close_today - close_yest) / close_yest

        if pct > NEUTRAL_BENCHMARK:
            return Trend.UP
        elif pct < -NEUTRAL_BENCHMARK:
            return Trend.DOWN
        else:
            return Trend.NEUTRAL


    def next(self):
        state = self.get_state()

        self.position.close()
        
        if state == trend_down:
            self.sell()
        elif state == trend_up:
            self.buy()

        

In [523]:
bt = Backtest(test_data, MarkovStrategy, cash=10000, commission=0)
stats = bt.run()
stats

Start                     2019-12-31 00:00:00
End                       2023-11-28 00:00:00
Duration                   1428 days 00:00:00
Exposure Time [%]                    88.22335
Equity Final [$]                  1943.333128
Equity Peak [$]                   85646.58388
Return [%]                         -80.566669
Buy & Hold Return [%]              -95.833333
Return (Ann.) [%]                   -34.23669
Volatility (Ann.) [%]              131.024536
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -98.296463
Avg. Drawdown [%]                    -26.7216
Max. Drawdown Duration     1012 days 00:00:00
Avg. Drawdown Duration       88 days 00:00:00
# Trades                                  716
Win Rate [%]                         42.73743
Best Trade [%]                      78.571437
Worst Trade [%]                    -88.888882
Avg. Trade [%]                    

In [524]:
bt.plot()


/home/hunter/math214/214finalproj/.env/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/hunter/math214/214finalproj/.env/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/hunter/math214/214finalproj/.env/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/home/hunter/math214/214finalproj/.env/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p10832', ...)

In [525]:
NAIVE_PROFIT = (10000 / test_data["Adj Close"][0]) * test_data["Adj Close"][-1]
print(NAIVE_PROFIT)

416.66665631863845


/tmp/ipykernel_8099/2454357074.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  NAIVE_PROFIT = (10000 / test_data["Adj Close"][0]) * test_data["Adj Close"][-1]


In [526]:
bt